# IMPORTING LIBs

In [5]:
import pandas as pd
import boto3
import json
import psycopg2


# 1. Configuring credentials

In [6]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY = config.get('AWS','KEY')
SECRET = config.get('AWS','SECRET')

DWH_ENDPOINT = config.get("DWH","DWH_ENDPOINT")
DWH_IAM_ROLE = config.get("DWH","DWH_IAM_ROLE")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")
DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")


(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)


# 2. Creating clients

In [7]:
import boto3
redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )
s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

# 3. Checking bucket

In [ ]:
sampleDbBucket =  s3.Bucket("udacity-dend")
for obj in sampleDbBucket.objects.filter(Prefix="song_data"):
    print(obj)

# 4. Connecting to Redshift (my account)

In [8]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [9]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)
%sql $conn_string

'Connected'

# 5. Dropping tables (if they existed) and creating tables needed

In [ ]:
staging_events_table_drop = "DROP TABLE IF EXISTS staging_events"
staging_songs_table_drop = "DROP TABLE IF EXISTS staging_songs"
songplay_table_drop = "DROP TABLE IF EXISTS songplays"
user_table_drop = "DROP TABLE IF EXISTS users"
song_table_drop = "DROP TABLE IF EXISTS songs"
artist_table_drop = "DROP TABLE IF EXISTS artists"
time_table_drop = "DROP TABLE IF EXISTS time"

%sql $staging_events_table_drop
%sql $staging_songs_table_drop
%sql $songplay_table_drop
%sql $user_table_drop
%sql $song_table_drop
%sql $artist_table_drop
%sql $time_table_drop

### Creating the tables I'll need

In [10]:
%%sql
CREATE TABLE IF NOT EXISTS staging_events (artist text distkey, 
                                           auth text, 
                                           first_name text, 
                                           gender text, 
                                           item_in_session text,
                                           last_name text,
                                           leng float, 
                                           level text, 
                                           location text, 
                                           method text, 
                                           page text,
                                           registration float, 
                                           session_id int, 
                                           song text, 
                                           status int, 
                                           ts timestamp,
                                           user_agent text, 
                                           user_id int);

CREATE TABLE IF NOT exists staging_songs (artist_name text distkey,
                                          artist_id text,
                                          artist_latitude float,
                                          artist_longitude float,
                                          artist_location text,
                                          song_id text,
                                          title text,
                                          duration float,
                                          year int,
                                          num_songs int);

CREATE TABLE IF NOT EXISTS songplays (songplay_id int identity(0,1) primary key,
                                     start_time timestamp NOT NULL,
                                     user_id text NOT NULL,
                                     level text NOT NULL,
                                     song_id text NOT NULL,
                                     artist_id text NOT NULL distkey,
                                     session_id int NOT NULL,
                                     location text NOT NULL,
                                     user_agent text NOT NULL);

CREATE TABLE IF NOT EXISTS users (user_id int primary key,
                                  first_name text NOT NULL,
                                  last_name text NOT NULL,
                                  gender text NOT NULL,
                                  level text sortkey NOT NULL);

CREATE TABLE IF NOT EXISTS songs (song_id text primary key,
                                  title text NOT NULL,
                                  artist_id text sortkey NOT NULL,
                                  year int NOT NULL,
                                  duration float NOT NULL);

CREATE TABLE IF NOT EXISTS artists(artist_id text PRIMARY KEY sortkey,
                                   artist_name text NOT NULL,
                                   location text,
                                   latitude float,
                                   longitude float);

CREATE TABLE IF NOT EXISTS time(start_time timestamp PRIMARY KEY sortkey,
                                hour int NOT NULL,
                                day int NOT NULL,
                                week int NOT NULL,
                                month int NOT NULL,
                                year int NOT NULL,
                                weekday int NOT NULL);

Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

# 6. Copying from s3 to staging

In [11]:
staging_events_copy = ("""copy staging_events from {} 
                        credentials 'aws_iam_role={}'
                        compupdate off region 'us-west-2' 
                        FORMAT AS JSON {}
                        TIMEFORMAT as 'epochmillisecs'""").format(config['S3']['LOG_DATA'],
                                                                  config['DWH']['DWH_IAM_ROLE'],
                                                                  config['S3']['LOG_JSONPATH'])
                                                           
%sql $staging_events_copy

Done.


[]

In [12]:
staging_songs_copy = ("""copy staging_songs from {}
                      credentials 'aws_iam_role={}'
                      compupdate off region 'us-west-2' 
                      FORMAT AS JSON 'auto'""").format(config['S3'].get('SONG_DATA'),
                                                config['DWH'].get('DWH_IAM_ROLE').strip("'"))
%sql $staging_songs_copy

Done.


[]

# 7. Loading new tables

## Inserting in fact table 
songplays - records in event data associated with song plays i.e. records with page NextSong

In [13]:
# FINAL TABLES

songplay_table_insert = ("""INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
                            SELECT 
                            (events.ts)              AS start_time, 
                            events.user_id           AS user_id, 
                            events.level             AS level,
                            songs.song_id            AS song_id, 
                            songs.artist_id          AS artist_id, 
                            events.session_id        AS session_id, 
                            events.location          AS location, 
                            events.user_agent        AS user_agent
                            FROM staging_events events 
                            JOIN staging_songs songs ON 
                            events.artist = songs.artist_name AND events.song = songs.title
                            WHERE events.page = 'NextSong' """)

%sql $songplay_table_insert

333 rows affected.


[]

In [14]:
user_table_insert = ("""INSERT INTO users (user_id, first_name, last_name, gender, level)
                        SELECT DISTINCT
                        user_id,
                        first_name,
                        last_name,
                        gender,
                        level
                        FROM staging_events events
                        WHERE user_id is NOT NULL""")
%sql $user_table_insert

105 rows affected.


[]

In [15]:
song_table_insert = ("""INSERT INTO songs (song_id, title, artist_id, year, duration)
                     SELECT DISTINCT 
                     song_id, 
                     title, 
                     artist_id, 
                     year, 
                     duration
                     FROM staging_songs
                     WHERE song_id IS NOT NULL""")

%sql $song_table_insert

14896 rows affected.


[]

In [16]:
artist_table_insert = ("""INSERT INTO artists (artist_id, artist_name, location, latitude, longitude)
                       SELECT DISTINCT 
                       artist_id, 
                       artist_name, 
                       artist_location,
                       artist_latitude, 
                       artist_longitude
                       FROM staging_songs
                       WHERE artist_id IS NOT NULL""")

%sql $artist_table_insert

10025 rows affected.


[]

In [17]:
time_table_insert = ("""INSERT INTO time (start_time, hour, day,week, month, year, weekday)
                     SELECT DISTINCT 
                     ts                        AS start_time, 
                     extract(hour from ts)     AS hour, 
                     extract(day from ts)      AS day,
                     extract(week from ts)     AS week,
                     extract(month from ts)    AS month,
                     extract(year from ts)     AS year, 
                     extract(weekday from ts)  AS weekday
                     FROM staging_events WHERE ts IS NOT NULL""")
%sql $time_table_insert

8023 rows affected.


[]

# 8. ANALYTICS

### 1 - Top 5 played songs 

In [18]:
%%sql

SELECT title, 
COUNT(songplays.song_id) AS plays
FROM songplays
JOIN songs on songs.song_id = songplays.song_id
GROUP BY title
ORDER BY plays DESC
LIMIT 5;

5 rows affected.


title,plays
You're The One,37
I CAN'T GET STARTED,9
Catch You Baby (Steve Pitron & Max Sanna Radio Edit),9
Nothin' On You [feat. Bruno Mars] (Album Version),8
Hey Daddy (Daddy's Home),6


### 2 - Top 5 played artists 

In [19]:
%%sql

SELECT artist_name, 
COUNT(songplays.artist_id) AS artists
FROM songplays
JOIN artists on artists.artist_id = songplays.artist_id
GROUP BY artist_name
ORDER BY artists DESC
LIMIT 5;

5 rows affected.


artist_name,artists
Dwight Yoakam,37
Kid Cudi / Kanye West / Common,10
Kid Cudi,10
Ron Carter,9
Lonnie Gordon,9


### 3 - Artists who have more songs in Sparkify

In [20]:
%%sql

SELECT DISTINCT 
artist_name,
COUNT(song_id) AS number_of_songs
FROM songs 
JOIN artists on songs.artist_id = artists.artist_id
GROUP BY artist_name
ORDER BY number_of_songs desc
LIMIT 5
                     

5 rows affected.


artist_name,number_of_songs
Polygon Window,9
Badly Drawn Boy,9
Alison Krauss / Union Station,9
Radiohead,8
The Breeders,8


### 4 - Most used devices

In [21]:
%%sql

SELECT user_agent, 
COUNT(songplays.user_agent) AS devices
FROM songplays
GROUP BY user_agent
ORDER BY devices DESC
LIMIT 5;

5 rows affected.


user_agent,devices
"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",51
Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0,43
"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2""",34
"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36""",33
"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",23


# 9. Disconnecting

In [22]:
staging_events_table_drop = "DROP TABLE IF EXISTS staging_events"
staging_songs_table_drop = "DROP TABLE IF EXISTS staging_songs"
songplay_table_drop = "DROP TABLE IF EXISTS songplays"
user_table_drop = "DROP TABLE IF EXISTS users"
song_table_drop = "DROP TABLE IF EXISTS songs"
artist_table_drop = "DROP TABLE IF EXISTS artists"
time_table_drop = "DROP TABLE IF EXISTS time"

%sql $staging_events_table_drop
%sql $staging_songs_table_drop
%sql $songplay_table_drop
%sql $user_table_drop
%sql $song_table_drop
%sql $artist_table_drop
%sql $time_table_drop

Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]